In [1]:
import sys

sys.path.append("../")

import sqlite3
import pandas as pd
from functions.env import DB_SCIENCE_PATH_NEW

data_path = "/Users/charlesdedampierre/Desktop/cultura_project/cultura.db"

conn = sqlite3.connect(data_path)

In [18]:
df_ind = pd.read_sql("SELECT * FROM individual_occupations", conn)

df_ind_science = df_ind[df_ind["occupations_category"].str.contains("science")]
list_individuals = list(set(df_ind_science.individual_wikidata_id))

df_ind = df_ind[df_ind['individual_wikidata_id'].isin(list_individuals)]

df_ind_country = pd.read_sql("SELECT * FROM individuals_main_information", conn)
df_ind_country = df_ind_country[['individual_wikidata_id', 'country_code', 'birthyear']].dropna().drop_duplicates()
df_ind_country = df_ind_country[df_ind_country['birthyear']<=1700]
df_ind_country = pd.merge(df_ind_country, df_ind, on = 'individual_wikidata_id')
df_ind_country = df_ind_country[['individual_wikidata_id', 'country_code']].drop_duplicates().reset_index(drop=True)
df_ind_country

,individual_wikidata_id,country_code
0,Q22691480,DEU
1,Q80665311,NLD
2,Q1881341,IRQ
3,Q983482,ISL
4,Q214080,CHL
...,...,...
5877,Q64016,AGO
5878,Q18945731,FRA
5879,Q65921411,LTU
5880,Q158097,DEU


In [19]:
import geopandas as gpd
df_world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
df_world = df_world[['iso_a3', 'continent']].drop_duplicates()
df_world = df_world.rename(columns = {'iso_a3':'country_code'})

df_final = pd.merge(df_ind_country, df_world, on = 'country_code')
df_final.continent.value_counts()

/var/folders/lx/9b13r6ln63jb54_3xlf9b_380000gn/T/ipykernel_8070/3108450217.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  df_world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))


Europe           4850
Asia              515
North America     213
South America     183
Africa            120
Oceania             1
Name: continent, dtype: int64

In [31]:
df_continent = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
df_continent[df_continent['continent']=='Asia']


/var/folders/lx/9b13r6ln63jb54_3xlf9b_380000gn/T/ipykernel_8070/495868171.py:1: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  df_continent = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))


,pop_est,continent,name,iso_a3,gdp_md_est,geometry
5,1.851393e+07,Asia,Kazakhstan,KAZ,181665,"POLYGON ((87.35997 49.21498, 86.59878 48.54918..."
6,3.358065e+07,Asia,Uzbekistan,UZB,57921,"POLYGON ((55.96819 41.30864, 55.92892 44.99586..."
8,2.706256e+08,Asia,Indonesia,IDN,1119190,"MULTIPOLYGON (((141.00021 -2.60015, 141.01706 ..."
24,1.293119e+06,Asia,Timor-Leste,TLS,2017,"POLYGON ((124.96868 -8.89279, 125.08625 -8.656..."
76,9.053300e+06,Asia,Israel,ISR,394652,"POLYGON ((35.71992 32.70919, 35.54567 32.39399..."
77,6.855713e+06,Asia,Lebanon,LBN,51991,"POLYGON ((35.82110 33.27743, 35.55280 33.26427..."
79,4.685306e+06,Asia,Palestine,PSE,16276,"POLYGON ((35.39756 31.48909, 34.92741 31.35344..."
83,1.010169e+07,Asia,Jordan,JOR,44502,"POLYGON ((35.54567 32.39399, 35.71992 32.70919..."
84,9.770529e+06,Asia,United Arab Emirates,ARE,421142,"POLYGON ((51.57952 24.24550, 51.75744 24.29407..."
85,2.832067e+06,Asia,Qatar,QAT,175837,"POLYGON ((50.81011 24.75474, 50.74391 25.48242..."


In [27]:
df_continent = df_final[['country_code', 'continent']].drop_duplicates()
df_continent = df_continent.sort_values('continent')
df_continent[df_continent['continent']=='Asia']

,country_code,continent
5802,PAK,Asia
5241,KOR,Asia
5793,IDN,Asia
1206,IRQ,Asia
5752,MYS,Asia
5499,YEM,Asia
5733,PSE,Asia
5731,MMR,Asia
3426,TUR,Asia
5695,UZB,Asia
